## Import Libraries
Let's import our fpl_draft_league tool and alias it as fpl. 

In [1]:
from etl_scripts.api import get_dataframe
from lib.charts import get_matches_stacked, get_points_over_time, get_streaks

## Inspecting the Data

Using `utils.get_dataframes(json_file)` we can pull 3 useful dataframes! 
* League entries
* Matches
* Current standings

In [2]:
league_entry_df = get_dataframe('league_entries')
matches_df = get_dataframe('matches')
standings_df = get_dataframe('standings')

The league entries dataframe contains all league participants, with some IDs, names and waiver picks. Most useful bit here is probably a lookup between names, team names and ids. Also the waiver pick may be interesting to compare to performance!!

In [3]:
league_entry_df

,entry_id,entry_name,id,joined_time,player_first_name,player_last_name,short_name,waiver_pick
0,301781,AFC Richmond,303734,2023-08-06T20:41:50.138791Z,Michael,Sparre,MS,1
1,301945,Misery Loves Kompany,303901,2023-08-06T20:47:39.198833Z,Bryce,Allred,BA,2
2,301962,Hwanging and banging,303918,2023-08-06T20:48:15.247736Z,Rory,McGinnis,RM,3
3,301967,relegation parade,303924,2023-08-06T20:48:29.893578Z,Josh,Gumacal,JG,4
4,302050,Kante stole my wife,304012,2023-08-06T20:51:50.893775Z,Jack,Thurber,JT,5
5,309454,Kloppenheimer FC,311625,2023-08-07T06:34:03.772588Z,Vedant,Sahu,VS,6
6,341081,StevieG08,343640,2023-08-08T03:11:18.954889Z,Christian,Pinho,CP,7
7,341436,dirty Mike&the boyz,344002,2023-08-08T03:42:04.013511Z,Jackson,Nagle,JN,8


The standings dataframe is again quite obvious, a row for each team and their points, their score, their rank. Cool. The only thing is that this is a "BC" view, (Business Current)... it would be cool to see the rankings over time so you can see movers and shakers.

In [4]:
standings_df

,last_rank,league_entry,matches_drawn,matches_lost,matches_played,matches_won,points_against,points_for,rank,rank_sort,total
0,None,303734,0,0,0,0,0,0,None,None,0
1,None,303901,0,0,0,0,0,0,None,None,0
2,None,303918,0,0,0,0,0,0,None,None,0
3,None,303924,0,0,0,0,0,0,None,None,0
4,None,304012,0,0,0,0,0,0,None,None,0
5,None,311625,0,0,0,0,0,0,None,None,0
6,None,343640,0,0,0,0,0,0,None,None,0
7,None,344002,0,0,0,0,0,0,None,None,0


The matches dataframe has every match, including unplayed matches and details about who played who, who scored and so on. The `winning_league_entry` and `winning_method` are all "None" so I'm not exactly sure what this is. 

In [5]:
matches_df

,event,finished,league_entry_1,league_entry_1_points,league_entry_2,league_entry_2_points,started,winning_league_entry,winning_method
0,1,False,311625,33,303734,34,True,None,None
1,1,False,343640,17,304012,64,True,None,None
2,1,False,344002,24,303924,53,True,None,None
3,1,False,303901,28,303918,22,True,None,None
4,2,False,303734,0,303901,0,False,None,None
...,...,...,...,...,...,...,...,...,...
147,37,False,304012,0,311625,0,False,None,None
148,38,False,343640,0,303734,0,False,None,None
149,38,False,344002,0,311625,0,False,None,None
150,38,False,303901,0,304012,0,False,None,None


## Standings Over Time

The first thing I want to explore is league standings over time (week by week). 

I realise that with all of the match data in `matches_df` I can essentially rebuild the history of standings. The only tricky thing is that the `matches_df` is a row per matchup, not a row per team's match. This makes it difficult to plot because I basically need a row by row of team, week, result.

The `fpl.get_points_over_time` function will basically produce a row per team's match, and then produce a plot of the standings over time for you.

In [6]:
stacked_df = get_matches_stacked(matches_df, league_entry_df)

In [7]:
stacked_df

,match,team,score,points,margin
0,1,Vedant,33,0,-1
1,1,Christian,17,0,-47
2,1,Jackson,24,0,-29
3,1,Bryce,28,3,6
4,1,Michael,34,3,1
5,1,Jack,64,3,47
6,1,Josh,53,3,29
7,1,Rory,22,0,-6


## Streaks
The next thing I want to explore are winning streaks.
* Who holds the record?!
* Who is someone to watch out for on a hot current streak?

In [8]:
df = get_streaks(stacked_df)
df.head()

,,match,team,score,points,margin,binary,streak
0,3,1,Bryce,28,3,6,1,1
1,1,1,Christian,17,0,-47,-1,-1
2,5,1,Jack,64,3,47,1,1
3,2,1,Jackson,24,0,-29,-1,-1
4,6,1,Josh,53,3,29,1,1


### What are people's record streaks?

In [10]:
df[['team', 'streak']].groupby(['team'], as_index=False).max().sort_values(by='streak', ascending=False)


,team,streak
0,Bryce,1
2,Jack,1
4,Josh,1
5,Michael,1
1,Christian,-1
3,Jackson,-1
6,Rory,-1
7,Vedant,-1


### Who's on the hot streak now?

In [11]:
df[df['match'] == df.match.max()].sort_values(by='streak', ascending=False)

,,match,team,score,points,margin,binary,streak
0,3,1,Bryce,28,3,6,1,1
2,5,1,Jack,64,3,47,1,1
4,6,1,Josh,53,3,29,1,1
5,4,1,Michael,34,3,1,1,1
1,1,1,Christian,17,0,-47,-1,-1
3,2,1,Jackson,24,0,-29,-1,-1
6,7,1,Rory,22,0,-6,-1,-1
7,0,1,Vedant,33,0,-1,-1,-1


In [30]:
matches_group = stacked_df.groupby(['match'])
# matches_group = stacked_df.groupby(['match']).apply(lambda x: x)

In [32]:
matches_group.groups

{1: [0, 1, 2, 3, 4, 5, 6, 7]}

In [16]:
gw_highscores = stacked_df.iloc[matches_group['score'].idxmax()]

In [17]:
gw_highscores

,match,team,score,points,margin
5,1,Jack,64,3,47


In [20]:
gw_highscores[['team','score']].groupby('team').count().sort_values(by='score', ascending=False)

,score
team,
Jack,1


In [21]:
def find_highscores(group):
    
    group['gw_highscore_index'] = group['score'].idxmax()
    
    return group

In [33]:
df = find_highscores(gw_highscores)

/var/folders/5z/w7_q1_p97k15691qskxrkzr40000gn/T/ipykernel_50030/1796700211.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['gw_highscore_index'] = group['score'].idxmax()


In [34]:
df

,match,team,score,points,margin,gw_highscore_index
5,1,Jack,64,3,47,5
